In [19]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import time

In [20]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [21]:
#Define database and collection
db = client.mars_db
collection = db.latest_news

In [22]:
# def init_browser():
# Replace the path with your actual path to the chromedriver
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [23]:
# Get the title and paragraph content of the latest news story
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(2)
html = browser.html
soup = BeautifulSoup(html, "html.parser")
#print(soup.prettify())

In [24]:
test = soup.select_one("ul.item_list li.slide")
#print(test.prettify())

In [25]:
news_title = soup.find('div', class_="bottom_gradient")
news_title.text

'Mars Is Getting a New Robotic Meteorologist'

In [26]:
news_p = soup.find('div', class_="article_teaser_body")
news_p.text

"Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles."

In [27]:
# Get the JPL Mars Space Images - featured image url by clicking full image and then more info
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
full_image = browser.find_by_id("full_image")
full_image.click()
more_info = browser.find_link_by_partial_text("more info")
more_info.click()

In [28]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [29]:
result = soup.find('figure', class_ = "lede")
#print(result.prettify())

In [30]:
featured_image_url = result.find('a')["href"]
featured_image_url = f"https://www.jpl.nasa.gov{featured_image_url}"
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA15283_hires.jpg'

In [31]:
# Get table containing facts from the Mars Facts webpage. Convert data to a HTML table string
import pandas as pd
url = "https://space-facts.com/mars/"
facts_df = pd.read_html(url)[0]
#facts_df


In [32]:
facts_df = facts_df.rename(columns={0:"Description", 1:"Data"})
facts_df

,Description,Data
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [33]:
facts_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Data</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium B

In [34]:
# Scrape the USGS Astrogeology site for titles and images of Mars four hemispheres.
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(2)
html = browser.html
soup = BeautifulSoup(html, "html.parser")



In [35]:
test_1 = soup.find_all('div', class_ = "item")
#print(test_1)

result = []


for item in test_1:
    title = item.find('h3').text
    
    url_test =  item.find('a')['href']
    url_1 = f"https://astrogeology.usgs.gov{url_test}"
    browser.visit(url_1)
    time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
   # print(soup.prettify())
    image_url = soup.find("div", class_ = "downloads")
    #print(image_url)
    url = image_url.find('a')["href"]
    result.append({"title": title,  "img_url": url})
    
    #print(result)

In [36]:
result

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [37]:
cerberus = result[0]
cerberus

{'title': 'Cerberus Hemisphere Enhanced',
 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}

In [38]:
schiaparelli = result[1]
schiaparelli

{'title': 'Schiaparelli Hemisphere Enhanced',
 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}

In [39]:
syrtis = result[2]
syrtis

{'title': 'Syrtis Major Hemisphere Enhanced',
 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}

In [40]:
valles = result[3]
valles

{'title': 'Valles Marineris Hemisphere Enhanced',
 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}

In [41]:
# post = {
#     'title': news_title,
#     'paragraph': news_p,
#     'image': featured_image_url,
#     'facts': facts_df,
#     'hemispheres': result
# }

# collection.insert_one(post)

InvalidDocument: cannot encode object: <div class="bottom_gradient"><div><h3>Mars Is Getting a New Robotic Meteorologist</h3></div></div>, of type: <class 'bs4.element.Tag'>

In [17]:
# post

NameError: name 'post' is not defined